<a href="https://colab.research.google.com/github/gtoubian/cce/blob/main/Project_3_Gabriel_Toubian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Project 3: NHL Player Salaries**
**By: Gabriel Toubian**

This project will involve the yearly salaries for NHL players and the different factors that influence them. We can make inferences that determine what makes a player more "valuable" to an NHL team (earn a higher salary).

**Importing the dataset and building a pandas dataframe out of it**

In [3]:
from google.colab import files
data = files.upload()

Saving nhl.csv to nhl (1).csv


In [7]:
import pandas as pd

import io
nhl = pd.read_csv('nhl.csv', encoding = "ISO-8859-1")

In [10]:
nhl = nhl.dropna(how='any')

In [14]:
nhl.tail()

,Salary,Born,City,Pr/St,Cntry,Nat,Ht,Wt,DftYr,DftRd,Ovrl,Hand,Last Name,First Name,Position,Team,GP,G,A,A1,A2,PTS,+/-,E+/-,PIM,Shifts,TOI,TOIX,TOI/GP,TOI/GP.1,TOI%,IPP%,SH%,SV%,PDO,F/60,A/60,Pct%,Diff,Diff/60,...,Min,Maj,Match,Misc,Game,CF,CA,FF,FA,SF,SA,xGF,xGA,SCF,SCA,GF,GA,RBF,RBA,RSF,RSA,DSF,DSA,FOW,FOL,HF,HA,GVA,TKA,PENT,PEND,OPS,DPS,PS,OTOI,Grit,DAP,Pace,GS,GS/G
606,680000,93-04-16,Burnsville,MN,USA,USA,77,210,2011.0,4.0,101.0,L,Labate,Joseph,C,VAN,13,0,0,0,0,0,-2,-1.3,21,147,5276,88.0,6.77,6.77,13.1,0.0,6.3,0.909,972.0,1.36,2.73,33.3,-2,-1.36,...,3,3,0,0,0,68.0,89.0,47.0,66.0,32.0,44.0,2.3,3.2,8.0,7.0,2.0,4.0,0.0,5.0,6.0,5.0,6,10,35.0,35.0,54.0,42.0,13.0,11.0,9.0,4.0,-0.2,0.0,-0.2,581.67,49,8.7,107.1,-1.2,-0.09
607,600000,92-09-26,Denver,CO,USA,USA,73,201,2011.0,3.0,82.0,R,Shore,Nick,C,L.A,70,6,11,6,5,17,-2,4.2,20,1258,53712,894.6,12.78,12.78,22.8,65.4,6.6,0.920,985.0,1.74,2.01,46.4,-4,-0.27,...,10,0,0,0,0,780.0,764.0,569.0,569.0,395.0,373.0,32.9,33.6,106.0,106.0,26.0,30.0,33.0,20.0,38.0,38.0,71,58,383.0,382.0,439.0,429.0,113.0,48.0,55.0,47.0,0.0,1.3,1.3,3023.36,100,6.9,103.6,23.2,0.33
608,9000000,85-01-21,Madison,WI,USA,USA,74,206,2003.0,1.0,7.0,L,Suter,Ryan,D,MIN,82,9,31,10,21,40,34,19.7,36,2319,132469,2190.7,26.93,26.86,44.7,34.5,10.2,0.927,1029.0,3.18,2.14,59.8,38,1.04,...,18,0,0,0,0,2056.0,2009.0,1546.0,1510.0,1137.0,1073.0,107.8,82.6,361.0,244.0,116.0,78.0,110.0,50.0,98.0,52.0,208,102,1257.0,1122.0,513.0,583.0,198.0,247.0,95.0,112.0,2.7,6.6,9.3,2715.54,213,6.3,111.3,44.1,0.54
609,4250000,87-03-30,Montreal,QC,CAN,CAN,73,205,2005.0,2.0,35.0,L,Vlasic,Marc-Edouard,D,S.J,75,6,22,14,8,28,4,0.6,35,1982,95555,1573.0,21.07,21.09,35.5,43.8,9.2,0.908,1000.0,2.44,2.52,49.2,-2,-0.08,...,10,1,0,0,1,1334.0,1519.0,980.0,1038.0,693.0,714.0,60.1,65.0,196.0,215.0,64.0,66.0,42.0,42.0,60.0,71.0,102,113,739.0,833.0,426.0,513.0,311.0,189.0,67.0,75.0,1.7,4.1,5.8,2855.83,200,4.1,108.8,32.5,0.43
611,925000,93-05-12,St. Albert,AB,CAN,CAN,78,226,2012.0,3.0,86.0,R,Parayko,Colton,D,STL,81,4,31,17,14,35,7,1.7,32,2365,103018,1712.4,21.20,21.18,35.4,46.1,9.2,0.917,1009.0,2.66,2.24,54.3,12,0.42,...,16,0,0,0,0,1507.0,1462.0,1127.0,1081.0,826.0,771.0,65.8,66.0,208.0,222.0,76.0,64.0,45.0,43.0,62.0,54.0,107,97,813.0,815.0,578.0,642.0,150.0,174.0,100.0,100.0,2.2,4.5,6.7,3119.42,229,7.4,104.0,45.8,0.56


We can see this dataset includes the salaries and stats for 611 players.

# Hypothesis testing

Whether a player is left or right handed can have on impact on the type of position they can play optimally. Therefore, do players of a certain handedness come at a premium? Let's start by seeing if whether a player is **left or right handed** has an impact on their salary. We can run a one-way ANOVA test to see if this factor is statistically significant.

In [20]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
import scipy.stats as sc

In [22]:
nhlhand = nhl[["Hand", "Salary"]]
Left = np.array(nhlhand[nhlhand["Hand"]=="L"]["Salary"])
Right = np.array(nhlhand[nhlhand["Hand"]=="R"]["Salary"])

sc.f_oneway(Left, Right)

F_onewayResult(statistic=0.03681398777778426, pvalue=0.8479533459419761)

With a p-value of well over 0.05, we cannot reject the null hypothesis, which means that a player's handedness doesn't necessarily play a sigificant part in determining their salary.

How about a player's **Nationality**? Do certain countries have a tendency of producing more "valuable" players through their programs? Let's take 2 countries that have a tendency to finish relatively high in country-based competitions: **Canada and USA**

In [58]:
nhlnat = nhl[["Nat", "Salary"]]

In [60]:
nhlnat = nhlnat[(nhlnat.Nat == 'CAN') | (nhlnat.Nat == 'USA')]

In [63]:
Canadian = np.array(nhlnat[nhlnat["Nat"]=="CAN"]["Salary"])
American = np.array(nhlnat[nhlnat["Nat"]=="USA"]["Salary"])

sc.f_oneway(Canadian, American)

F_onewayResult(statistic=0.019945699195699655, pvalue=0.8877687260535069)

With a p-value of well over 0.05, we cannot reject the null hypothesis, which means that a player's nationality (in this case, whether they are Canadian or American) doesn't necessarily play a sigificant part in determining their salary.

# Multiple Regression

In [71]:
nhl2=nhl.copy()

In [72]:
del nhl2['Salary']
del nhl2['Born']
del nhl2['City']
del nhl2['Pr/St']
del nhl2['Cntry']
del nhl2['Nat']
del nhl2['DftYr']
del nhl2['Hand']
del nhl2['Last Name']
del nhl2['First Name']
del nhl2['Position']
del nhl2['Team']

In [74]:
X = nhl2
y = nhl['Salary']
sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 Salary   R-squared (uncentered):                   0.904
Model:                            OLS   Adj. R-squared (uncentered):              0.848
Method:                 Least Squares   F-statistic:                              16.01
Date:                Fri, 23 Apr 2021   Prob (F-statistic):                    7.72e-69
Time:                        16:44:00   Log-Likelihood:                         -5493.7
No. Observations:                 359   AIC:                                  1.125e+04
Df Residuals:                     226   BIC:                                  1.177e+04
Df Model:                         133                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Ht         -1.049e+05   6.03e+04     -1.740      0.083   -2.24e+05    1.39e+04
Wt          2.711e+04   9264.395      2.926      0.004    8856.513    4.54e+04
DftRd      -1.762e+05   3.08e+05     -0.572      0.568   -7.83e+05    4.31e+05
Ovrl        4083.7337   9921.463      0.412      0.681   -1.55e+04    2.36e+04
GP         -7.935e+04   9.09e+04     -0.873      0.384   -2.58e+05    9.98e+04
G          -2.674e+06   3.09e+06     -0.865      0.388   -8.76e+06    3.41e+06
A          -9.474e+06   2.78e+06     -3.412      0.001   -1.49e+07      -4e+06
A1          5.755e+06   2.17e+06      2.657      0.008    1.49e+06       1e+07
A2          5.715e+06   2.16e+06      2.645      0.009    1.46e+06    9.97e+06
PTS         3.899e+06   1.92e+06      2.032      0.043    1.19e+05    7.68e+06
+/-        -2902.2398    3.1e+04     -0.094      0.926    -6.4e+04    5.82e+04
E+/-        4.434e+04   4.79e+04      0.925      0.356   -5.01e+04    1.39e+05
PIM         1.181e+04    9.6e+04      0.123      0.902   -1.77e+05    2.01e+05
Shifts     -3462.9948   1476.993     -2.345      0.020   -6373.433    -552.557
TOI         -417.0925    556.409     -0.750      0.454   -1513.506     679.321
TOIX        3.197e+04   3.33e+04      0.960      0.338   -3.37e+04    9.76e+04
TOI/GP      1.198e+06   2.34e+06      0.513      0.609    -3.4e+06     5.8e+06
TOI/GP.1    -8.99e+05   2.34e+06     -0.384      0.702   -5.52e+06    3.72e+06
TOI%       -1.628e+05   1.69e+05     -0.966      0.335   -4.95e+05    1.69e+05
IPP%       -4034.9291   5179.287     -0.779      0.437   -1.42e+04    6170.940
SH%         7.482e+05    1.9e+06      0.394      0.694   -2.99e+06    4.49e+06
SV%         7.055e+07   1.89e+08      0.374      0.709   -3.01e+08    4.42e+08
PDO        -6.772e+04   1.89e+05     -0.359      0.720    -4.4e+05    3.04e+05
F/60       -2.205e+07    1.8e+07     -1.223      0.223   -5.76e+07    1.35e+07
A/60        2.186e+07    1.8e+07      1.212      0.227   -1.37e+07    5.74e+07
Pct%       -6336.6469   9267.814     -0.684      0.495   -2.46e+04    1.19e+04
Diff       -1.647e+04   1.78e+04     -0.926      0.355   -5.15e+04    1.86e+04
Diff/60     2.184e+07    1.8e+07      1.210      0.227   -1.37e+07    5.74e+07
iCF        -1.834e+05    7.5e+04     -2.444      0.015   -3.31e+05   -3.55e+04
iCF.1       1.866e+05   7.65e+04      2.440      0.015    3.59e+04    3.37e+05
iFF         7.268e+05   3.29e+05      2.211      0.028    7.92e+04    1.37e+06
iSF        -1.037e+06   4.19e+05     -2.476      0.014   -1.86e+06   -2.12e+05
iSF.1       2.768e+05   8.36e+05      0.331      0.741   -1.37e+06    1.92e+06
iSF.2      -1.708e+06   1.97e+06     -0.866      0.388    -5.6e+06    2.18e+06
ixG        -1255.2677   2.74e+05     -0.005      0.996    -5.